This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests, json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Add api key and limit parameters to base url. 
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY, 'limit': 1}
response = requests.get(url, params=params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# Print json response. 
response.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).**

In [5]:
# Set the appropriate parameters to retrieve data for only 2017. 
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}

# Retrieve json data. 
data17 = requests.get(url, params=params)

**2. Convert the returned JSON object into a Python dictionary.**

In [6]:
# Convert response data to dictionary. 
data17 = data17.json()

**3. Calculate what the highest and lowest opening prices were for the stock in this period.**

In [7]:
# Helper function to find values of desired feature. 
def find_values(feature): 
    idx = data17['dataset_data']['column_names'].index(feature)
    return [day[idx] for day in data17['dataset_data']['data'] if day[idx] != None]

In [8]:
opening = find_values('Open')  # Opening values. 
lowest_opening = min(opening)   # Lowest opening value. 
highest_opening = max(opening)  # Highest opening value. 

print(f'Lowest opening: {lowest_opening:.2f}\nHighest opening: {highest_opening:.2f}')

Lowest opening: 34.00
Highest opening: 53.11


**4. What was the largest change in any one day (based on High and Low price)?**

In [9]:
high = find_values('High')  # High values. 
low = find_values('Low')    # Low values. 

# Calculate the differences for each day. 
diffs = [high - low for high, low in zip(high, low)]
maxdiff = max(diffs)

print(f'Max. difference between high and low for any day: {maxdiff:.2f}')

Max. difference between high and low for any day: 2.81


**5. What was the largest change between any two days (based on Closing Price)?**

It was unclear to me whether this questions is asking for the max. difference between any two days or between any two consecutive days. The first cell is the answer to the former and the second to the latter. 

In [10]:
closing = find_values('Close')  # Closing values. 

# Calculate the difference between max and min value. 
close_max_diff = max(closing) - min(closing)

print(f'Max. difference of closing between any two days: {close_max_diff:.2f}')

Max. difference of closing between any two days: 19.03


In [11]:
# Calculate the max difference between consecutive opening days. 
close_max_diff = max([day1 - day0 for day0, day1 in zip(closing[:-1], closing[1:])])

print(f'Max. difference of closing between any two consecutive days: {close_max_diff:.2f}')

Max. difference of closing between any two consecutive days: 2.56


**6. What was the average daily trading volume during this year?**

In [12]:
trading_volume = find_values('Traded Volume')  # Traded volume values. 

# Calculate the mean. 
avg = sum(trading_volume) / len(trading_volume)

print(f'Mean traded volume for 2017: {avg:.2f}')

Mean traded volume for 2017: 89124.34


**7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)**

In [13]:
tv_sorted = sorted(trading_volume)  # Sort the trading volume values. 

# There are two scenarios: 
# 1. Length of list is odd and the median is the value in the middle. 
# 2. Length of list is even and the median is the average of the two middle values. 
length = len(tv_sorted)
if length % 2 == 1: 
    idx = length // 2 
    median = tv_sorted[idx]
elif length % 2 == 0: 
    idx = length // 2
    median = (tv_sorted[idx - 1] + tv_sorted[idx]) / 2
    
print(f'Median traded volume for 2017: {median:.2f}')

Median traded volume for 2017: 76286.00
